In [12]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import binned_statistic_2d

In [19]:
# базовые вещи
dataFolder = 'raw_data'

# Координаты и сетка Нью-Йорка
ymin = 40.49612
ymax = 40.91553
xmin = -74.25559
xmax = -73.70001
split_n = 50
step_x = (xmax - xmin) / split_n
step_y = (ymax - ymin) / split_n

In [20]:
# функция получения региона
def get_region(coord):   
    x = int((coord[0] - xmin) / step_x)
    y = int((coord[1] - ymin) / step_y)
    if (x == 0):
        dLon = x + 1
    return x * split_n + y + 1

In [21]:
regions = pd.read_csv("raw_data/regions.csv", sep = ';')
regions.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [22]:
def SkipMinSec(timestamp):
    return np.datetime64(timestamp, 'h')

def prepare_data(df):  
   
    #извлекаем имена используемых колонок по их индексам, т.к. их названия меняются в разных файлах
    pickupTimeCol = rawData.columns[1]
    dropoffTimeCol = rawData.columns[2]
    passengerCountCol = rawData.columns[3]
    tripDistanceCol = rawData.columns[4]
    pickupLongitudeCol = rawData.columns[5]
    pickupLatitudeCol = rawData.columns[6]
    
    #преобразуем колонки времени начала и окончания поездки в формат datetime64
    rawData[pickupTimeCol] = pd.to_datetime(rawData[pickupTimeCol])
    rawData[dropoffTimeCol] = pd.to_datetime(rawData[dropoffTimeCol])
    
    df = df.drop(df[df.tpep_pickup_datetime == df.tpep_dropoff_datetime].index, axis = 0)
    df = df.drop(df[df.passenger_count == 0].index, axis = 0)
    df = df.drop(df[df.trip_distance == 0].index, axis = 0)
    df = df.drop(df[(df.pickup_latitude < ymin)  | (df.pickup_latitude > ymax)].index, axis = 0)
    df = df.drop(df[(df.pickup_longitude < xmin) | (df.pickup_longitude > xmax)].index, axis = 0)
   
    #определяем район для каждой записи
    df['region'] = df[[pickupLongitudeCol,pickupLatitudeCol]].apply(get_region, axis = 1)
    
    #удаляем минуты и секунды из времени начала поездки
    df[pickupTimeCol] = df[pickupTimeCol].apply(SkipMinSec)
    
    #время-регион
    return df[pickupTimeCol].values.astype(np.int64), df.region.values, df.shape[0] 

In [23]:
filename = 'yellow_tripdata_2016-05.csv'
FileredDataLen = 0
print ("Обработка началась ", datetime.datetime.now())  

#for filename in rawDataFiles:
    
#будем читать записи порциями
rawDataReader = pd.read_csv(dataFolder + '/' + filename, chunksize=100000)

#создаем список меток времени по количеству часов в текущем месяце
currentDate = filename.replace('yellow_tripdata_','').replace('.csv','')
currentDate = pd.to_datetime(currentDate)
# для зануления в тех ячейках, в которых не было поездок
timestamps = pd.date_range(currentDate, periods = currentDate.days_in_month*24, freq='H')

#массив для накопления статистики
cummulativeStat = np.zeros([timestamps.shape[0], regions.shape[0]])

for rawData in rawDataReader:        
    x,y, dfshape = prepare_data(rawData)   
 
    FileredDataLen += dfshape
    #первый день следующего месяца
    additionalHour = (timestamps[-1] + datetime.timedelta(hours=1)).to_datetime64().astype(np.int64)
    xBins = np.hstack([timestamps.astype(np.int64), additionalHour])
    yBins = np.hstack([regions.region, 2501])

    cummulativeStat += binned_statistic_2d(x, y, None, 'count', [xBins, yBins]).statistic

#Сохраняем результат
resultTable = pd.DataFrame(data = cummulativeStat, index = timestamps, columns = regions.region, dtype=int)
resultTable.to_csv('processed_data/' + filename.replace('yellow_tripdata_','trip_count_'))
print ("%s: Файл %s обработан."%(datetime.datetime.now(), filename))
print ("Количество записей после фильтрации: ",FileredDataLen)
    
print ("Обработка завершена в ", datetime.datetime.now())

Обработка началась  2019-05-20 19:45:31.809248


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


2019-05-20 19:51:13.226711: Файл yellow_tripdata_2016-05.csv обработан.
Количество записей после фильтрации:  11626521
Обработка завершена в  2019-05-20 19:51:13.226711


In [24]:
resultTable.head()

region,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
2016-05-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
%%time
zeroVals=0
for i in range(1,2501):
    zeroVals += resultTable[resultTable[i] == 0].shape[0]
        
print ("Количество нулевых пар: ", zeroVals, " из ", resultTable.size)

Количество нулевых пар:  1718238  из  1860000
Wall time: 7.23 s
